In [27]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np 
import tensorflow as tf
from utils import *

#For visualizations
import matplotlib.pyplot as plt 
import seaborn as sns
sns.set(style = 'whitegrid', context='notebook')
%matplotlib notebook

import random as rn

RANDOM_SEED = 42

np.random.seed(RANDOM_SEED)
rn.seed(RANDOM_SEED)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Data Loading

In [2]:
raw_df = pd.read_csv('data\creditcard.csv')
print(f"Shape of dataset : {raw_df.shape}")

Shape of dataset : (284807, 31)


In [3]:
# add a negligible amount to avoid taking the log of 0
raw_df['log10_amount'] = np.log10(raw_df.Amount + 0.00001)

In [4]:
# manual parameter 
RATIO_TO_FRAUD = 15

#dropping redundant columns
raw_df = raw_df.drop(['Amount','Time'], axis=1)

# splitting by class
fraud = raw_df[raw_df.Class == 1]
clean = raw_df[raw_df.Class == 0]

# undersample clean transactions
clean_undersampled = clean.sample(
    int(len(fraud) * RATIO_TO_FRAUD),
    random_state=RANDOM_SEED
)

# concatenate with fraud transactions into a single dataframe
visualisation_initial = pd.concat([fraud, clean_undersampled])
column_names = list(visualisation_initial.drop('Class', axis=1).columns)

# isolate features from labels 
features, labels = visualisation_initial.drop('Class', axis=1).values, \
                   visualisation_initial.Class.values

In [5]:
print("Shape of clean transactions:", clean.shape)
print("Shape of Fraud transactions:", fraud.shape)
print("Clean Undersampled for t-SNE", clean_undersampled.shape)

Shape of clean transactions: (284315, 30)
Shape of Fraud transactions: (492, 30)
Clean Undersampled for t-SNE (7380, 30)


In [6]:
print(f"""The non-fraud dataset has been undersampled from {len(clean):,} to {len(clean_undersampled):,}.
This represents a ratio of {RATIO_TO_FRAUD}:1 to fraud.""")

The non-fraud dataset has been undersampled from 284,315 to 7,380.
This represents a ratio of 15:1 to fraud.


In [7]:
tsne_scatter(features, labels, dimensions=2, save_as='plots/tsne_initial_2d.png')

<IPython.core.display.Javascript object>

## Train- Validate-Test split
### Train data : transactions that were normal
### Validation data : transaction that were normal
### Test data = normal + fraud

In [8]:
print(f"""Shape of the datasets:
    clean (rows, cols) = {clean.shape}
    fraud (rows, cols) = {fraud.shape}""")

Shape of the datasets:
    clean (rows, cols) = (284315, 30)
    fraud (rows, cols) = (492, 30)


In [9]:
TRAINING_SAMPLE = 200000
VALIDATE_SIZE = 0.2

# shuffle our training set
clean = clean.sample(frac=1).reset_index(drop=True)

# training set: exlusively non-fraud transactions
X_train = clean.iloc[:TRAINING_SAMPLE].drop('Class', axis=1)

# testing  set: the remaining non-fraud + all the fraud 
X_test = pd.concat([clean.iloc[TRAINING_SAMPLE:], fraud]).sample(frac=1)

In [10]:
print(f"""Our testing set is composed as follows:

{X_test.Class.value_counts()}""")

Our testing set is composed as follows:

Class
0    84315
1      492
Name: count, dtype: int64


In [11]:
from sklearn.model_selection import train_test_split

# train // validate - no labels since they're all clean anyway
X_train, X_validate = train_test_split(X_train, 
                                       test_size=VALIDATE_SIZE, 
                                       random_state=RANDOM_SEED)

# manually splitting the labels from the test df
X_test, y_test = X_test.drop('Class', axis=1).values, X_test.Class.values

In [12]:
print(f"""Shape of the datasets:
    training (rows, cols) = {X_train.shape}
    validate (rows, cols) = {X_validate.shape}
    holdout  (rows, cols) = {X_test.shape}""")

Shape of the datasets:
    training (rows, cols) = (160000, 29)
    validate (rows, cols) = (40000, 29)
    holdout  (rows, cols) = (84807, 29)


In [13]:
X_train_transformed, X_validate_transformed,X_test_transformed = transform_data(X_train, X_validate ,X_test)

e:\SEM-3\Deep Learning\AnomalyDetectionAutoEncoders\.venv\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but Normalizer was fitted with feature names
  warnings.warn(


In [15]:
g = sns.PairGrid(X_train.iloc[:,:3].sample(600, random_state=RANDOM_SEED))
plt.subplots_adjust(top=0.9)
g.fig.suptitle('Before:')
g.map_diag(sns.kdeplot)
g.map_offdiag(sns.kdeplot)
plt.show()
plt.savefig('plots/KDE plot Before transformations')

<IPython.core.display.Javascript object>

In [16]:
g = sns.PairGrid(pd.DataFrame(X_train_transformed, columns=column_names).iloc[:,:3].sample(600, random_state=RANDOM_SEED))
plt.subplots_adjust(top=0.9)
g.fig.suptitle('After:')
g.map_diag(sns.kdeplot)
g.map_offdiag(sns.kdeplot)
plt.show()
plt.savefig('plots/KDE plot After transformations')

<IPython.core.display.Javascript object>

## Autoencoder and hyperparameters


In [17]:
input_dim = X_train_transformed.shape[1]
BATCH_SIZE = 256
EPOCHS = 100

In [18]:
autoencoder = tf.keras.models.Sequential([
    
    # deconstruct / encode
    tf.keras.layers.Dense(input_dim, activation='elu', input_shape=(input_dim, )), 
    tf.keras.layers.Dense(16, activation='elu'),
    tf.keras.layers.Dense(8, activation='elu'),
    tf.keras.layers.Dense(4, activation='elu'),
    tf.keras.layers.Dense(2, activation='elu'),
    
    # reconstruction / decode
    tf.keras.layers.Dense(4, activation='elu'),
    tf.keras.layers.Dense(8, activation='elu'),
    tf.keras.layers.Dense(16, activation='elu'),
    tf.keras.layers.Dense(input_dim, activation='elu')
    
])

In [19]:
autoencoder.compile(optimizer="adam", 
                    loss="mse",
                    metrics=["acc"])

In [20]:
autoencoder.summary();

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 29)                870       
                                                                 
 dense_1 (Dense)             (None, 16)                480       
                                                                 
 dense_2 (Dense)             (None, 8)                 136       
                                                                 
 dense_3 (Dense)             (None, 4)                 36        
                                                                 
 dense_4 (Dense)             (None, 2)                 10        
                                                                 
 dense_5 (Dense)             (None, 4)                 12        
                                                                 
 dense_6 (Dense)             (None, 8)                 4

## Callbacks

In [21]:
from datetime import datetime

# current date and time
yyyymmddHHMM = datetime.now().strftime('%Y%m%d%H%M')

# new folder for a new run
log_subdir = f'{yyyymmddHHMM}_batch{BATCH_SIZE}_layers{len(autoencoder.layers)}'

# define our early stopping
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0.0001,
    patience=10,
    verbose=1, 
    mode='min',
    restore_best_weights=True
)

save_model = tf.keras.callbacks.ModelCheckpoint(
    filepath='autoencoder_best_weights.hdf5',
    save_best_only=True,
    monitor='val_loss',
    verbose=0,
    mode='min'
)

tensorboard = tf.keras.callbacks.TensorBoard(
    f'logs/{log_subdir}',
    batch_size=BATCH_SIZE,
    update_freq='batch'
)

# callbacks argument only takes a list
cb = [early_stop, save_model, tensorboard]

In [22]:
history = autoencoder.fit(
    X_train_transformed, X_train_transformed,
    shuffle=True,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=cb,
    validation_data=(X_validate_transformed, X_validate_transformed)
);

Epoch 1/100


625/625 [==============================] - 5s 6ms/step - loss: 0.0308 - acc: 0.2297 - val_loss: 0.0171 - val_acc: 0.2609
Epoch 2/100
 23/625 [>.............................] - ETA: 2s - loss: 0.0171 - acc: 0.2541

e:\SEM-3\Deep Learning\AnomalyDetectionAutoEncoders\.venv\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


625/625 [==============================] - 3s 5ms/step - loss: 0.0164 - acc: 0.2579 - val_loss: 0.0151 - val_acc: 0.2785
Epoch 3/100
625/625 [==============================] - 3s 5ms/step - loss: 0.0146 - acc: 0.3327 - val_loss: 0.0145 - val_acc: 0.3363
Epoch 4/100
625/625 [==============================] - 3s 5ms/step - loss: 0.0144 - acc: 0.3403 - val_loss: 0.0142 - val_acc: 0.3322
Epoch 5/100
625/625 [==============================] - 3s 5ms/step - loss: 0.0140 - acc: 0.3342 - val_loss: 0.0139 - val_acc: 0.3351
Epoch 6/100
625/625 [==============================] - 3s 5ms/step - loss: 0.0138 - acc: 0.3289 - val_loss: 0.0138 - val_acc: 0.3272
Epoch 7/100
625/625 [==============================] - 3s 5ms/step - loss: 0.0137 - acc: 0.3349 - val_loss: 0.0137 - val_acc: 0.3353
Epoch 8/100
625/625 [==============================] - 3s 5ms/step - loss: 0.0136 - acc: 0.3478 - val_loss: 0.0135 - val_acc: 0.3496
Epoch 9/100
625/625 [==============================] - 3s 5ms/step - loss: 0.0133

In [23]:
# pass the transformed test set through the autoencoder to get the reconstructed result
reconstructions = autoencoder.predict(X_test_transformed)

2651/2651 [==============================] - 5s 2ms/step


In [24]:
# calculating the mean squared error reconstruction loss per row in the numpy array
mse = np.mean(np.power(X_test_transformed - reconstructions, 2), axis=1)

In [25]:
clean = mse[y_test==0]
fraud = mse[y_test==1]

fig, ax = plt.subplots(figsize=(6,6))

ax.hist(clean, bins=50, density=True, label="clean", alpha=.6, color="green")
ax.hist(fraud, bins=50, density=True, label="fraud", alpha=.6, color="red")

plt.title("(Normalized) Distribution of the Reconstruction Loss")
plt.legend()
plt.show()
plt.savefig('plots/Distribution of the Reconstruction Loss')

<IPython.core.display.Javascript object>

In [34]:
THRESHOLD = 2.5
z_scores = mad_score(mse)
outliers = z_scores > THRESHOLD

In [35]:
print(f"Detected {np.sum(outliers):,} outliers in a total of {np.size(z_scores):,} transactions [{np.sum(outliers)/np.size(z_scores):.2%}].")

Detected 2,874 outliers in a total of 84,807 transactions [3.39%].


In [36]:
from sklearn.metrics import (confusion_matrix, 
                             precision_recall_curve)

# get (mis)classification
cm = confusion_matrix(y_test, outliers)

# true/false positives/negatives
(tn, fp, 
 fn, tp) = cm.flatten()

In [37]:
print(f"""The classifications using the MAD method with threshold={THRESHOLD} are as follows:
{cm}

% of transactions labeled as fraud that were correct (precision): {tp}/({fp}+{tp}) = {tp/(fp+tp):.2%}
% of fraudulent transactions were caught succesfully (recall):    {tp}/({fn}+{tp}) = {tp/(fn+tp):.2%}""")

The classifications using the MAD method with threshold=2.5 are as follows:
[[81840  2475]
 [   93   399]]

% of transactions labeled as fraud that were correct (precision): 399/(2475+399) = 13.88%
% of fraudulent transactions were caught succesfully (recall):    399/(93+399) = 81.10%
